# 透過 Python 擷取網頁上的資料

# 新的 WiFi

CSIE_summer

pass: 33664888

在大數據變成顯學的時代，今天我們要做資料分析，若沒有大量的資料，是無法分析出任何有價值的資訊出來的。

但是在一般情況下，我們都沒有大量的資料可以做分析，但幸運的是，由於網路的蓬勃發展，獲取任何一個領域的資料比起過往相對就變得容易許多。但是，由於網頁上的資料非常多，若透過手動的方式截取資料，就不是一個很有效率的方法了。

因此這時候，透過程式自動從網路上蒐集資料這個問題也變得更加重要，只需一個簡單的程式，就能透過低成本並且自動化的方式從網頁上獲得大量的資料，因此**學習與實作網頁爬蟲成為一個投資報酬率極高的事務**，而 Python 語言由於生態系龐大，套件衆多，也讓用 Python 實作爬蟲比起其他語言相對簡單許多。

## 請先開啓範例網頁：

http://pythonscraping.com/pages/warandpeace.html

*以上練習用網頁由 Ryan Mitchell 維護，可以的話，大家買一本他的書支持他繼續維護這個網站：[連結](https://www.books.com.tw/products/0010800965)

在進入網頁之後，試試看直接點擊網頁，然後右鍵 -> 儲存，接下來瀏覽器會將一個副檔名為 **html** 的檔案存下來。由此我們發現，任何一個我們看得見的網頁，其實都是一個副檔名為 **.html** 的檔案，問題是，這個 **html** 檔案是從何而來？它的原理又是什麽？

## 網頁開發基礎： HTTP 溝通協定

HTTP 用白話講，就像是**電腦與電腦之間的共同語言**，電腦需要通過這個共同語言，才能在網絡上面互相溝通。

![](https://drive.google.com/uc?export=download&id=1UiDUXriZVo83ePdiSQTLS4QWnCN_YsVu)

而一個 HTTP 的請求 (Request) 是根據 **HTTP 動詞** (HTTP Verb) 以及**網址**才能運作，舉例來説，當你在輸入 https://www.google.com/ ，連至 Google 的首頁時，整個背後的運作流程是：

1. 用戶端 (你的瀏覽器) 針對 Google 的雲端伺服器發送了一個 GET Request
2. 而 Google 的雲端伺服器在收到請求後，將需要呈現該網頁的資料都計算完成
3. 接著 Google 的雲端伺服器會回傳一個回應 (Response)，這個 Response 内通常就包含了一個 html 檔案
4. 用戶端 (你的瀏覽器) 在下載了請求回傳的 html 檔案之後，將 html 程式碼渲染成網頁，呈現給使用者

## HTTP Request 的種類

一般來説，Request 有 GET 與 POST：

- GET 代表我需要查詢 / 顯示資料，像是 GET https://www.facebook.com/ 代表查詢 facebook 首頁
- POST 代表我需要新增資料，通常用於網頁上的表單，像是 POST https://www.foodpanda.com/orders 代表新增訂單

## HTTP response 的種類

- 一般伺服器通常是回傳 html 網頁檔案
- 但若是一些功能像是下載 / 輸出報表，Response 則是一個 xlsx / csv 檔案


## 從 Excel 的角度來理解...

像是 FB, Google, Yahoo 等網站，**其實都像是一個個運行在雲端上的 Excel 函數**。

而要使用該函數時，就必須透過一個 HTTP 的請求 (Request) 來呼叫該函數。讓瀏覽器發送 HTTP Request 的方法就是輸入網址，就像是在 Excel 上輸入公式一樣。

而該公式若執行成功，不同於 Excel 是將結果顯示在工作表上，HTTP Request 的結果就是一個 Response，該 Response 通常是顯示在瀏覽器上的網頁，也有可能是 xlsx / csv 檔案

<h1>War and Peace</h1> 

<p></p>


## 用 Python 實作爬蟲

Python 用來實作爬蟲的兩個主流套件：

- BeautifulSoup [官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)

- PyQuery [官方文件](https://pythonhosted.org/pyquery/)

簡單來説：

**BeautifulSoup** 是比較貼近**程式設計師**的角度去思考

**PyQuery** 貼近**網頁開發者**的角度去思考

在這堂課考量到並不是每一個人都具備網頁開發的背景，因此我們會使用 BeautifulSoup 套件來實作爬蟲

## 網頁開發 101

任何網頁都是由 **html 標籤(tag)** 所組成，基本結構如下
```html
<標籤名稱 class="類別名稱">內容</標籤名稱>
<標籤名稱 id="id名稱">內容</標籤名稱>
```
今天我們要擷取的任何內容，一定是被包裹在在某一個標籤裡面
而今天若網頁開發者需要改變任何一個標籤的**樣式**，就需要用到 **css** 語法
以上面的網頁為例，人名都是以綠色顯示，所以就先宣告一個名為 **green** 的 css 類別:

```html
<style>
.green{
	color:#55ff55;
}
</style>
```

若今天希望讓一個標籤的内容文字變成綠色，可以使用定義好的 .green 這個 css 類別：

```html
<span class="green">Prince Vasili Kuragin</span>
```

*想了解更多 html 可以看一下 Mozilla 官網的教學：[HTML 基礎](https://developer.mozilla.org/zh-TW/docs/Learn/Getting_started_with_the_web/HTML_basics)

## 使用 標籤 「類別名稱」取得資料

要擷取資料前，首先需要透過方法**選擇**到該標籤
##  BeautifulSoup 套件

典故來自 Alice in WonderLand 裏面一首同名的詩，由假海龜 (Mock Turtle) 所唱，影射英國料理假海龜湯...

有興趣自己可以去 Google, 不多説了...

想查看 BeautifulSoup 套件的功能請看一下官方的中文文件：

[BeautifulSoup 官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)


```python
from bs4 import BeautifulSoup
import requests

# 針對網頁發送 GET Reqeust
res = requests.get("http://pythonscraping.com/pages/warandpeace.html")
# 將回傳的 Response 内的文字用 BeautifulSoup 解析
html = BeautifulSoup(res.text, "html.parser")
```

In [1]:
import bs4
import requests

res = requests.get("http://pythonscraping.com/pages/warandpeace.html")

res.text

'<html>\n<head>\n<style>\n.green{\n\tcolor:#55ff55;\n}\n.red{\n\tcolor:#ff5555;\n}\n#text{\n\twidth:50%;\n}\n</style>\n</head>\n<body>\n<h1>War and Peace</h1>\n<h2>Chapter 1</h2>\n<div id="text">\n"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don\'t tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy \'faithful slave,\' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.</span>"\n<p/>\nIt was in July, 1805, and the speaker was the well-known <span class="green">Anna\nPavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya\nFedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man\nof high rank

In [2]:
html = bs4.BeautifulSoup(res.text, "html.parser")
html

<html>
<head>
<style>
.green{
	color:#55ff55;
}
.red{
	color:#ff5555;
}
#text{
	width:50%;
}
</style>
</head>
<body>
<h1>War and Peace</h1>
<h2>Chapter 1</h2>
<div id="text">
"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>"
<p></p>
It was in July, 1805, and the speaker was the well-known <span class="green">Anna
Pavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya
Fedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man
of high rank and importance, who was the firs

In [8]:
span_list = html.findAll("span", { "class": "green" })
span_list[2]

<span class="green">Prince Vasili Kuragin</span>

In [9]:
tag = span_list[2]
tag.text

'Prince Vasili Kuragin'

In [10]:
for tag in span_list:
    print(tag.text)
    print("=======")

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


## 另外再介紹一下 find() 方法...

```python
from bs4 import BeautifulSoup
import requests

res = requests.get("http://pythonscraping.com/pages/warandpeace.html")
html = BeautifulSoup(res.text, 'html.parser')
# 我們把該網頁使用 'green' css 類別的 span 標籤過濾出來...
name = html.find("span", {"class": "green"})
print(name)
```

# 網頁爬蟲實戰：(臺股爬蟲)

我們想要截取資料的網頁：[Yahoo Stock 台積電](https://tw.stock.yahoo.com/q/q?s=2330)

![](https://drive.google.com/uc?export=download&id=1samMUBW19ooC7UsntphFGG6QKjL5337R)

網址是：

https://tw.stock.yahoo.com/q/q?s=2330

```python
# 來試試看爬 yahoo stock 的網頁...
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
# 用 html 格式解碼 爬下來的檔案
html = BeautifulSoup(res.text, 'html.parser')
print(html)
```


In [12]:
from bs4 import BeautifulSoup
import requests

res = requests.get("https://tw.stock.yahoo.com/q/q?s=2330")
html = BeautifulSoup(res.text, "html.parser")
html

://tw.news.yahoo.com/weather/">氣象</a></li>
<li><a href="https://tw.sports.yahoo.com/">運動</a></li>
<li><a href="https://tw.video.yahoo.com/">Yahoo TV</a></li>
<li><a href="http://yahoomode.tumblr.com/yahooapp/">App下載</a></li>
<li><a href="https://tw.buy.yahoo.com/">購物中心</a></li>
<li><a href="https://tw.mall.yahoo.com/">商城</a></li>
<li><a href="https://tw.bid.yahoo.com/">拍賣</a></li>
<li>
<div class="more-anchor rapidnofollow" style="position: relative; z-index: 1; margin-left: -10px; outline: none; cursor: pointer;">
<div style="padding: 0 10px;">
<span>更多</span>
<svg data-icon="CoreArrowDown" height="8" style="margin-left: 5px; vertical-align: middle;" viewbox="0 0 512 512" width="8">
<path d="M500.77 131.432L477.53 108.18c-14.45-14.55-40.11-14.55-54.51 0L255.845 275.363 88.582 108.124c-15.015-14.874-39.363-14.874-54.42.108L10.94 131.486c-14.58 14.44-14.58 40.11-.033 54.442l217.77 217.845c15.004 14.82 39.33 14.874 54.42-.108L500.88 185.82c14.818-14.982 14.87-39.298-.11-54.388z"></path>


In [13]:
table = html.findAll("table", { "border": "2" })[0]
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
last_close = float(tds[7].text)
price_open = float(tds[8].text)
price_high = float(tds[9].text)
price_low = float(tds[10].text)
price_close = float(tds[3].text)

price_open, price_high, price_low, price_close, last_close

(440.0, 440.5, 431.0, 435.0, 444.0)

In [14]:
txt = html.findAll(text="個股資料")[0]
table = txt.parent.parent.parent
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
last_close = float(tds[7].text)
last_close

444.0

In [15]:
last_td = html.findAll("td", { "class": "tt"})[0]
price_low = last_td.find_previous_sibling("td").text
price_high = last_td.find_previous_sibling("td").find_previous_sibling("td").text
price_open = last_td.find_previous_sibling("td").find_previous_sibling("td").find_previous_sibling("td").text

price_open, price_high, price_low

('440.0', '440.5', '431.0')

# 問題是找到有用的資料如同大海撈針...

---
## 分析一下我們要爬的網頁

收盤價是被封裝在一個 **table** 標籤内部的一個 **td** 標籤


---
## table 標籤

一般網頁若要呈現重要的資訊，都會將資訊以網頁表格的形式呈現

今天若要利用 **html** 實作網頁上的表格，需要名爲 **table** 的標籤

換句話説，今天網頁上我們有興趣的資料，十之八九都是被封裝在 **table** 底下

把 **table** 的結構搞懂，就成了一件重要的事情。

---
# html table 標籤的結構

網頁上的資料大多都是匯整在表格、而 html 的表格則是由 table 標籤構成的：

![](https://drive.google.com/uc?export=download&id=1dmYx5qOD21tPWHATzIm6IxDnfO96sUac)

該 **table** 標籤内有兩個 **tr** 標籤

第二個 tr 標籤内的第八個 td 標簽是我們要的收盤價

![](https://drive.google.com/uc?export=download&id=109M71HQtR-kfZMklVit7AZRpIH4zhfe_)

---
# html 標簽的關聯

簡單來説，就是一個樹狀圖的概念：

![](https://drive.google.com/uc?export=download&id=1n2yZlQtj7EK2_NNIBXvEVGeEKBogfgYZ)

---
# 延申閲讀

HTML Table 教學

w3 school： [連結](https://www.w3schools.com/html/html_tables.asp)

Mozilla：[連結](https://developer.mozilla.org/zh-TW/docs/Web/HTML/Element/table)

---

## 開始實作台股爬蟲

```python
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
# 搜尋整個網頁裡的 table 標籤，將所有的表格讀取出來
html.findAll("table")
# 我們就發現網頁内有多個 table...
```

In [16]:
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
# 搜尋整個網頁裡的 table 標籤，將所有的表格讀取出來
html.findAll("table")

[<table border="0" cellpadding="1" cellspacing="0" width="750">
 <tr bgcolor="#FFF0C1">
 <td align="left" width="540">
 	     
 	</td>
 <td align="right" width="160"><font class="tt" color="#3333FF">　資料日期: 109/08/28</font></td>
 </tr>
 </table>, <table border="0" cellpadding="0" cellspacing="0" width="750">
 <tr>
 <td>
 <table border="2" width="750">
 <tr bgcolor="#fff0c1">
 <th align="center">股票<br/>代號</th>
 <th align="center" width="55">時間</th>
 <th align="center" width="55">成交</th>
 <th align="center" width="55">買進</th>
 <th align="center" width="55">賣出</th>
 <th align="center" width="55">漲跌</th>
 <th align="center" width="55">張數</th>
 <th align="center" width="55">昨收</th>
 <th align="center" width="55">開盤</th>
 <th align="center" width="55">最高</th>
 <th align="center" width="55">最低</th>
 <th align="center">個股資料</th>
 </tr>
 <tr>
 <td align="center" width="105"><a href="/q/bc?s=2330">2330台積電</a><br/><a href="/pf/pfsel?stocklist=2330;"><font size="-1">加到投資組合</font><br/></a></td>
 <td

## 檢查所有搜尋到的 table

仔細觀察一下，我們發現目標 table 的 **border** 屬性是被設定成 **2**

因此我們可以將程式碼修改爲...

```python
table = html.findAll("table", { "border": 2 })[0]
table
```

In [17]:
table = html.findAll("table", { "border": 2 })[0]
table

<table border="2" width="750">
<tr bgcolor="#fff0c1">
<th align="center">股票<br/>代號</th>
<th align="center" width="55">時間</th>
<th align="center" width="55">成交</th>
<th align="center" width="55">買進</th>
<th align="center" width="55">賣出</th>
<th align="center" width="55">漲跌</th>
<th align="center" width="55">張數</th>
<th align="center" width="55">昨收</th>
<th align="center" width="55">開盤</th>
<th align="center" width="55">最高</th>
<th align="center" width="55">最低</th>
<th align="center">個股資料</th>
</tr>
<tr>
<td align="center" width="105"><a href="/q/bc?s=2330">2330台積電</a><br/><a href="/pf/pfsel?stocklist=2330;"><font size="-1">加到投資組合</font><br/></a></td>
<td align="center" bgcolor="#FFFfff" nowrap="">14:30</td>
<td align="center" bgcolor="#FFFfff" nowrap=""><b>435.0</b></td>
<td align="center" bgcolor="#FFFfff" nowrap="">435.0</td>
<td align="center" bgcolor="#FFFfff" nowrap="">435.5</td>
<td align="center" bgcolor="#FFFfff" nowrap=""><font color="#009900">▽9.0
                <td align="ce

## 找尋 table 裡第二個 tr 標籤內所有的 td 標籤

```python
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
tds
```

In [18]:
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
tds

[<td align="center" width="105"><a href="/q/bc?s=2330">2330台積電</a><br/><a href="/pf/pfsel?stocklist=2330;"><font size="-1">加到投資組合</font><br/></a></td>,
 <td align="center" bgcolor="#FFFfff" nowrap="">14:30</td>,
 <td align="center" bgcolor="#FFFfff" nowrap=""><b>435.0</b></td>,
 <td align="center" bgcolor="#FFFfff" nowrap="">435.0</td>,
 <td align="center" bgcolor="#FFFfff" nowrap="">435.5</td>,
 <td align="center" bgcolor="#FFFfff" nowrap=""><font color="#009900">▽9.0
                 <td align="center" bgcolor="#FFFfff" nowrap="">56,689</td>
 <td align="center" bgcolor="#FFFfff" nowrap="">444.0</td>
 <td align="center" bgcolor="#FFFfff" nowrap="">440.0</td>
 <td align="center" bgcolor="#FFFfff" nowrap="">440.5</td>
 <td align="center" bgcolor="#FFFfff" nowrap="">431.0</td>
 <td align="center" class="tt" width="137">
 <a href="/q/ts?s=2330">成交彙整</a><br/><a href="/q/ta?s=2330">技術</a>　<a href="/q/h?s=2330">新聞</a><a href="/d/s/company_2330.html"><br/>基本</a>　<a href="/d/s/credit_2330.html

## 讀取出昨日收盤價

```python
last_close = tds[7].text
print(f"台積電今日收盤價：${last_close}")
```

In [19]:
last_close = tds[7].text
print(f"台積電今日收盤價：${last_close}")

台積電今日收盤價：$444.0


## 完成版網頁爬蟲

```python
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
table = html.findAll("table", { "border": 2 })[0]
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close = tds[7].text
print(f"台積電昨日收盤價：${last_close}")
```

In [20]:
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
table = html.findAll("table", { "border": 2 })[0]
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close = tds[7].text
print(f"台積電昨日收盤價：${last_close}")

台積電昨日收盤價：$444.0


## 解法二


```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
table = html.findAll(text='個股資料')[0].parent.parent.parent
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
data_row = table.findAll('tr')[1].findAll('td')
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_price = data_row[7].text
print(f"台積電昨日收盤價：${last_close}")
```

In [21]:
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
table = html.findAll(text='個股資料')[0].parent.parent.parent
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
data_row = table.findAll('tr')[1].findAll('td')
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_price = data_row[7].text
print(f"台積電昨日收盤價：${last_close}")

台積電昨日收盤價：$444.0


## 解法三...

![](https://drive.google.com/uc?export=download&id=1IJp5dze4stcwni0FNs2ThIjzYM_vYp4o)

注意 **tt** 是一個獨特的 css class，而 **tt** 只有被套用在最後一個 **td** 標籤上

```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 尋找 class 屬性為 tt 的 td 標籤
last_td = html.find("td", {"class": "tt"})
# find_previous_sibling('td') 代表尋找前一個 (左邊) td 標籤
last_close = last_td.find_previous_sibling('td').find_previous_sibling('td').find_previous_sibling('td').find_previous_sibling('td').text
print(f"台積電昨日收盤價：${last_close}")
```

In [22]:
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 尋找 class 屬性為 tt 的 td 標籤
last_td = html.find("td", {"class": "tt"})
# find_previous_sibling('td') 代表尋找前一個 (左邊) td 標籤
last_close = last_td.find_previous_sibling('td').find_previous_sibling('td').find_previous_sibling('td').find_previous_sibling('td').text
print(f"台積電昨日收盤價：${last_close}")

台積電昨日收盤價：$444.0


## 隨堂練習

請試試看從 Yahoo 股市網頁將 2330 的**開盤價、最高價、最低價**、以及**成交價**讀取出來：

```python
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
table = html.findAll("table", { "border": 2 })[0]
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close = tds[7].text
open_price = ________________
high_price = ________________
low_price = ________________
close_price = ________________
```

In [23]:
from bs4 import BeautifulSoup
import requests

res = requests.get("https://tw.stock.yahoo.com/q/q?s=2330")
html = BeautifulSoup(res.text, "html.parser")
table = html.findAll("table", { "border": "2" })[0]
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
last_close = float(tds[7].text)
price_open = float(tds[8].text)
price_high = float(tds[9].text)
price_low = float(tds[10].text)
price_close = float(tds[3].text)

price_open, price_high, price_low, price_close, last_close

(440.0, 440.5, 431.0, 435.0, 444.0)

## 將結果寫入 Excel

用 xlwings 開啓 **tw_stock_portfolio.xlsx**：

```python
import xlwings as xw
import time

wb = xw.Book(r"tw_stock_portfolio.xlsx")
sheet = wb.sheets["TW2330"]
```

偵測最後一個 row:

```python
last_row = sheet.range("A1").end("down").row
last_row
```

## 產生格式化的時間字串

```python
import time

time.strftime("%Y/%m/%d")
```

## 將時間與收盤價寫入 Excel 

```python
sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d")
sheet.range(f"B{last_row+1}").value = closing_price
```

# 完成版程式碼

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import xlwings as xw
# 截取台積電的資料 
res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
table = html.findAll("table", { "border": 2 })[0]
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
closing_close = tds[2].text
# 將台積電資料寫入 Excel
wb = xw.Book(r"檔案 tw_stock_portfolio.xlsx 的路徑")
date = time.strftime("%Y/%m/%d")
sheet = wb.sheets["TW2330"]
last_row = sheet.range("B1").end("down").row
sheet.range(f"B{last_row+1}").value = closing_price
sheet.range(f"A{last_row+1}").value = date

# 小結：

1. 學習與實作網頁爬蟲是一個**投資報酬率極高的事務**
2. Python 語言由於使用者衆多，**與爬蟲相關的套件、解決方案、與教學也多，讓實作變得相對簡單**
3. 實作上，最困難的部分在於**解析網頁的 html 結構**
4. 網頁的資料很大的機率都是被封裝在 **table** 這個 html 標簽下
5. 但若今天**網頁改版，原先寫好的爬蟲就有可能截取不到資料**

# 功課：匯率爬蟲

請寫一個網頁爬蟲，截取臺灣銀行牌告匯率網頁：

http://rate.bot.com.tw/xrt?Lang=zh-TW


並且將匯率資料用以下格式呈現在 Excel 内：
![](https://drive.google.com/uc?export=download&id=1YCl-QcAJCW951AhosB3HhuV7Fwy3hjMZ)